In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2 
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [ ]:
dataframe = pd.read_csv('../input/chinese-mnist/chinese_mnist.csv')
index = dataframe.iloc[:,:-2].values 
value = dataframe["code"].values

In [ ]:
print(index.shape)
print(len(value))

In [ ]:
filename_label_link = {}

filename_list = []
label_list = []
listImageArray = []
listImageNameArray = [] 
for i in range(0,len(value)):
    codeImage= index[i]
    filename = "input_%s_%s_%s.jpg" % (codeImage[0], codeImage[1], codeImage[2])
    val = value[i] 
    filename_label_link[filename] = val
    filename_list.append(filename)
    label_list.append(val)
    a = cv2.imread("../input/chinese-mnist/data/data/%s"%filename,0)
    listImageArray.append(a)
    listImageNameArray.append(val)

In [ ]:
listImageArray = np.array(listImageArray)
listImageNameArray = np.array(listImageNameArray)
print(listImageArray.shape)
print(listImageNameArray.shape)

In [ ]:
print("Before flatten:",listImageArray.shape)

listFlattened = []

# Duoi thang tung tam hinh ra 64x64 = 4096 cot
for i in range(len(listImageArray)):
    listFlattened.append(listImageArray[i].flatten()/255)


listFlattened = np.array(listFlattened)
listFlattenedScale = scale(listFlattened)

In [ ]:
print(listFlattenedScale.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
x_train, x_test, y_train, y_test = train_test_split(listFlattenedScale,listImageNameArray, test_size = 0.2, random_state = 10)



print("x_train: ",x_train.shape)
print("y_train: ",y_train.shape)
print("x_train[0] shape: ",x_train[0])
print("x_test: ",x_test.shape)
print("y_test: ",y_test.shape)
print("y_test[0] shape: ",y_test[0])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import operator 
from operator import itemgetter

In [ ]:
def euc_dist(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

In [ ]:
class KNN:
    def __init__(self, K=3):
        self.K = K
    def fit(self, x_train, y_train):
        self.X_train = x_train
        self.Y_train = y_train
    def predict(self, X_test):
        predictions = [] 
        for i in range(len(X_test)):
            dist = np.array([euc_dist(X_test[i], x_t) for x_t in   
            self.X_train])
            dist_sorted = dist.argsort()[:self.K]
            neigh_count = {}
            for idx in dist_sorted:
                if self.Y_train[idx] in neigh_count:
                    neigh_count[self.Y_train[idx]] += 1
                else:
                    neigh_count[self.Y_train[idx]] = 1
            sorted_neigh_count = sorted(neigh_count.items(),    
            key=operator.itemgetter(1), reverse=True)
            predictions.append(sorted_neigh_count[0][0]) 
        return predictions

In [ ]:
kVals = np.arange(3,100,2)
accuracies = []
for k in kVals:
  model = KNN(K = k)
  model.fit(x_train, y_train)
  pred = model.predict(x_test)
  acc = accuracy_score(y_test, pred)
  accuracies.append(acc)
  print("K = "+str(k)+"; Accuracy: "+str(acc))